In [1]:
using FFTW, DelimitedFiles, Printf

# 処理するフォルダのパス
folder_path = raw"C:\Users\sigot\OneDrive\デスクトップ\6.0Hz_8.0Lmin_C001H001S0002"
# 出力フォルダの基準パス
base_output_folder = raw"D:\新しいフォルダー (3)"
# テキストデータ間隔（秒）
interval_sec = 1/60
# バンドパスフィルタの閾値（Hz）
highpass_threshold = 0  # 低周波カット
lowpass_threshold = 5   # 高周波カット

# 指定フォルダ内のテキストファイルを取得
all_files = filter(x -> endswith(x, ".txt"), readdir(folder_path, join=true))

# 共通部分（ファイル名の末尾6桁を除いた部分）を取得
function get_common_prefix_and_suffix(filename)
    base = splitext(basename(filename))[1]  # 拡張子除去
    if length(base) > 6
        return base[1:end-6], base[end-5:end]  # (共通部分, 可変部分)
    else
        return "", base  # 例外処理
    end
end

# すべてのファイルの共通部分を取得し、グループ化
file_groups = Dict{String, Vector{Tuple{String, String}}}()
for file in all_files
    prefix, suffix = get_common_prefix_and_suffix(file)
    if prefix != ""
        if !haskey(file_groups, prefix)
            file_groups[prefix] = []
        end
        push!(file_groups[prefix], (file, suffix))
    end
end

# 1つの共通部分に対応するファイルのみ処理（最初のグループを採用）
first_group = first(values(file_groups))  # 先頭のグループを取得
sort!(first_group, by = x -> x[2])  # 可変部分（番号順）でソート
common_prefix = first(first_group)[1] |> get_common_prefix_and_suffix |> first  # 先頭ファイルの共通部分を取得

# フォルダ名を入力フォルダと同じ命名規則で作成（filteredなし）
folder_name = "hp$(highpass_threshold)_lp$(lowpass_threshold)_int$(round(interval_sec, digits=5))_$(common_prefix)"
output_folder = joinpath(base_output_folder, folder_name)

# フォルダが存在しない場合は作成
if !isdir(output_folder)
    mkdir(output_folder)
end

# データの格納用辞書
velocity_data = Dict()
coordinates = Set{Tuple{Float64, Float64}}()

timesteps = length(first_group)  # タイムステップ数

# 各ファイルを読み込み、速度データを格納
for (file, _) in first_group
    data = readdlm(file)
    for row in eachrow(data)
        x, y, vx, vy, speed = row[1:5]
        push!(coordinates, (x, y))
        if !haskey(velocity_data, (x, y))
            velocity_data[(x, y)] = Float64[]
        end
        push!(velocity_data[(x, y)], speed)
    end
end

# FFT、バンドパスフィルタ、逆FFTを適用
t_filtered = Dict()
fs = 1 / interval_sec  # サンプリング周波数
freqs = fftfreq(timesteps, fs)

for coord in coordinates
    vels = velocity_data[coord]
    fft_data = fft(vels)

    # バンドパスフィルタ適用
    fft_data[abs.(freqs) .< highpass_threshold] .= 0  # 低周波成分除去
    fft_data[abs.(freqs) .> lowpass_threshold] .= 0   # 高周波成分除去

    # 逆FFTで時間領域に戻す
    t_filtered[coord] = real(ifft(fft_data))
end

# フィルタ後のデータを新しいテキストファイルに出力（フォーマット維持）
for (i, (file, suffix)) in enumerate(first_group)
    data = readdlm(file)
    new_data = copy(data)
    for row in eachrow(new_data)
        x, y = row[1:2]
        row[5] = t_filtered[(x, y)][i]  # 速度値を更新
    end
    
    # 出力ファイル名を変更（filteredなし）
    output_filename = "hp$(highpass_threshold)_lp$(lowpass_threshold)_int$(round(interval_sec, digits=5))_$(common_prefix)_$(suffix).txt"
    output_file = joinpath(output_folder, output_filename)

    # テキストのフォーマットを統一（有効数字とスペース調整）
    open(output_file, "w") do f
        for row in eachrow(new_data)
            formatted_row = @sprintf("%4d %4d % .7e % .7e % .7e % .7e % .7e % .7e",
                                     row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8])
            println(f, formatted_row)
        end
    end
end

# 座標 (8,8) の処理結果をログ出力
if haskey(t_filtered, (8.0, 8.0))
    println("Original velocities at (8,8):", velocity_data[(8.0, 8.0)])
    println("Filtered velocities at (8,8):", t_filtered[(8.0, 8.0)])
end


Original velocities at (8,8):[4.6783028, 4.8932728, 0.16808899, 5.5555519, 3.1350697, 3.4941864, 3.8758931, 0.0, 4.2045929, 3.6928726, 3.3815724, 3.4227186, 0.0, 0.55316837, 2.7951131, 0.0, 0.0, 4.4217946, 2.9903205, 0.0, 2.9950037, 3.7936214, 0.065602696, 3.6017683, 3.2412159, 2.8750191, 3.2163197, 4.1911984, 2.5006645, 0.0, 0.0, 0.0, 4.2102298, 2.7624188, 0.0, 3.3820546, 3.1203962, 0.0, 4.1413993, 4.0242627, 3.6569499, 3.2866824, 3.5636663, 3.6139177, 0.0, 3.2838746, 3.1548495, 0.074668677, 0.0, 3.37413, 2.9012932, 2.9924676, 3.5157458, 0.0, 0.0, 3.3218989, 3.1220627, 4.2798806, 3.6808997, 3.2451097, 2.9378383, 0.0, 3.7893577, 3.8418187, 3.6792854, 3.9137259, 0.0, 0.0, 0.0, 0.0, 0.14079366, 2.2137937, 3.9646936, 2.5491172, 2.8750308, 2.9654356, 0.17608738, 3.5839711, 0.0, 0.0, 0.0, 0.0, 0.0, 0.091955551, 1.9533218, 3.1164989, 3.6134258, 0.0, 3.5992287, 3.1358426, 4.1301003, 0.0, 0.0, 0.0, 1.9258581, 3.0735517, 2.2191525, 0.0, 3.9542538, 3.7475323, 0.0, 0.21986474, 2.9602647, 3.294949